# MSM analysis

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

In this notebook, we will cover how to analyze an MSM and how the modeled processes correspond to MSM spectral properties.
We assume that you are familiar with data loading/visualization,
dimension reduction, and the estimation and validation process .


**Remember**:
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;
- you can find the full documentation at [PyEMMA.org](http://www.pyemma.org).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import mdshare
from tqdm.notebook import tqdm
import pyemma

## Case 1: preprocessed, two-dimensional data (toy model)
We load the two-dimensional trajectory from an archive using numpy,
directly discretize the full space using $k$-means clustering,
visualize the marginal and joint distributions of both components as well as the cluster centers,
and show the implied timescale (ITS) convergence:

In [ ]:
from deeptime.clustering import KMeans
from deeptime.markov.msm import MaximumLikelihoodMSM
from deeptime.plots import plot_implied_timescales
from deeptime.util.validation import implied_timescales

file = mdshare.fetch('hmm-doublewell-2d-100k.npz', working_directory='data')
with np.load(file) as fh:
    data = fh['trajectory']

cluster = KMeans(50).fit_fetch(data)
dtraj = cluster.transform(data)

models = []
for lag in [1, 2, 3, 5, 7, 10]:
    models.append(MaximumLikelihoodMSM(lagtime=lag).fit_fetch(dtraj))

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
pyemma.plots.plot_feature_histograms(data, feature_labels=['$x$', '$y$'], ax=axes[0])
pyemma.plots.plot_density(*data.T, ax=axes[1], cbar=False, alpha=0.1)
axes[1].scatter(*cluster.cluster_centers.T, s=15, c='C1')
axes[1].set_xlabel('$x$')
axes[1].set_ylabel('$y$')
axes[1].set_xlim(-4, 4)
axes[1].set_ylim(-4, 4)
axes[1].set_aspect('equal')
plot_implied_timescales(implied_timescales(models), ax=axes[2], n_its=4)
fig.tight_layout()

The plots show us the marginal (left panel) and joint distributions along with the cluster centers (middle panel).
The implied timescales are converged (right panel). 

Before we proceed, let's have a look at the implied timescales error bars.
They were computed from a Bayesian MSM, as requested by the `bayesian='True'` argument of the `implied_timescales_msm()` function.

As mentioned before, Bayesian MSMs incorporate a sample of transition matrices.
Target properties such as implied timescales can now simply be computed from the individual matrices.
Thereby, the posterior distributions of these properties can be estimated.
The ITS plot shows an interval that contains $95\%$ of the Bayesian samples.

In [ ]:
from deeptime.markov import TransitionCountEstimator
from deeptime.markov.msm import BayesianMSM

counts = TransitionCountEstimator(lagtime=1, count_mode="effective").fit_fetch(dtraj)
bayesian_msm = BayesianMSM().fit_fetch(counts)

For any PyEMMA method that derives target properties from MSMs, sample mean and confidence intervals (as defined by the function argument above) are directly accessible with  `mean` and `L` / `R`.
Further, `std` is available for computing the standard deviation.

In the more general case, it might be interesting to extract the full sample of a function evaluation with `evaluate_samples()`.
The syntax is equivalent for all those functions.

In [ ]:
sample_stats = bayesian_msm.gather_stats('timescales', k=1)

print(f'Mean of first ITS: {sample_stats.mean} ± {sample_stats.std}')
print(f'Confidence interval: [{sample_stats.L}, {sample_stats.R}]')

Please note that sample mean and maximum likelihood estimates are not identical and generally do not provide numerically identical results.

Now, for the sake of simplicity we proceed with the analysis of a maximum likelihood MSM.
We estimate it at lag time $1$ step...

In [ ]:
msm_estimator = MaximumLikelihoodMSM(lagtime=1)
msm = msm_estimator.fit_fetch(dtraj)

... and check for disconnectivity.
The MSM is constructed on the largest set of discrete states that are (reversibly) connected.
The `active_state_fraction` and `active_count_fraction` show us the fraction of discrete states and transition counts from our data which are part of this largest set and, thus, used for the model:

In [ ]:
print('fraction of states used = {:f}'.format(msm.state_fraction))
print('fraction of counts used = {:f}'.format(msm.count_fraction))

The fraction is, in both cases, $1$ and, thus, we have no disconnected states (which we would have to exclude from our analysis).

If there were any disconnectivities in our data (fractions $<1$),
we could access the indices of the **active states** (members of the largest connected set) via the `state_symbols` attribute:

In [ ]:
print(msm.count_model.state_symbols)

With this potential issue out of the way, we can extract our first (stationary/thermodynamic) property,
the `stationary_distribution`:

In [ ]:
print(msm.stationary_distribution)
print('sum of weights = {:f}'.format(msm.stationary_distribution.sum()))

The attribute `msm.stationary_distribution` tells us, for each discrete state, the absolute probability of observing said state in global equilibrium.
Mathematically speaking, the stationary distribution $\pi$ is the left eigenvector of the transition matrix $\mathbf{P}$ to the eigenvalue $1$:

$$\pi^\top \mathbf{P} = \pi^\top.$$

Please note that the $\pi$ is fundamentaly different from a normalized histogram of states:
for the histogram of states to accurately describe the stationary distribution, the data needs to be sampled from global equilibrium, i.e, the data points need to be statistically independent.
The MSM approach, on the other hand, only requires local equilibrium, i.e., statistical independence of state transitions.
Thus, the MSM approach requires a much weaker and, in practice, much easier to satisfy condition than simply counting state visits.

We can use the stationary distribution to, e.g., visualize the weight of the dicrete states and, thus, to highlight which areas of our feature space are most probable.
Here, we show all data points in a two dimensional scatter plot and color/weight them according to their discrete state membership:

In [ ]:
fig, ax, misc = pyemma.plots.plot_contour(
    *data.T, msm.stationary_distribution[dtraj],
    cbar_label='stationary distribution',
    method='nearest', mask=True)
ax.scatter(*cluster.cluster_centers.T, s=15, c='C1')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
ax.set_aspect('equal')
fig.tight_layout()

The stationary distribution can also be used to correct the `pyemma.plots.plot_free_energy()` function that we used to visualize this dataset.
This might be necessary if the data points are not sampled from global equilibrium.

In this case, we assign the weight of the corresponding discrete state to each data point and pass this information to the plotting function via its `weights` parameter:

In [ ]:
fig, ax, misc = pyemma.plots.plot_free_energy(
    *data.T,
    weights=np.concatenate(msm.compute_trajectory_weights(dtraj)),
    legacy=False)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
ax.set_aspect('equal')
fig.tight_layout()

We will see further uses of the stationary distribution later.
But for now, we continue the analysis of our model by visualizing its (right) eigenvectors which encode the dynamical processes.
First, we notice that the first right eigenvector is a constant $1$.

In [ ]:
eigvec = msm.eigenvectors_right()
print('first eigenvector is one: {} (min={}, max={})'.format(
    np.allclose(eigvec[:, 0], 1, atol=1e-15), eigvec[:, 0].min(), eigvec[:, 0].max()))

Second, the higher eigenvectors can be visualized as follows:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))
for i, ax in enumerate(axes.flat):
    pyemma.plots.plot_contour(
        *data.T, eigvec[dtraj, i + 1], ax=ax, cmap='PiYG',
        cbar_label='{}. right eigenvector'.format(i + 2), mask=True)
    ax.scatter(*cluster.cluster_centers.T, s=15, c='C1')
    ax.set_xlabel('$x$')
    ax.set_xlim(-4, 4)
    ax.set_ylim(-4, 4)
    ax.set_aspect('equal')
axes[0].set_ylabel('$y$')
fig.tight_layout()

The right eigenvectors can be used to visualize the processes governed by the corresponding implied timescales.
The first right eigenvector (always) is $(1,\dots,1)^\top$ for an MSM transition matrix and it corresponds to the stationary process (infinite implied timescale).

The second right eigenvector corresponds to the slowest process;
its entries are negative for one group of discrete states and positive for the other group.
This tells us that the slowest process happens between these two groups and that the process relaxes on the slowest ITS ($\approx 8.5$ steps).

The third and fourth eigenvectors show a larger spread of values and no clear grouping.
In combination with the ITS convergence plot, we can safely assume that these eigenvectors contain just noise and do not indicate any resolved processes.

We then continue to validate our MSM with a CK test for $2$ metastable states which are already indicated by the second right eigenvector.

In [ ]:
nstates = 2

models = [MaximumLikelihoodMSM(lagtime=lag).fit_fetch(dtraj) for lag in range(2, 10)]
ck_test = msm.ck_test(models, n_metastable_sets=nstates)

from deeptime.plots import plot_ck_test
plot_ck_test(ck_test);

We now save the model to do more analyses with PCCA++ and TPT in the next tutorial.

In [ ]:
import pickle

with open("nb4_cluster.pkl", "wb") as f:
    pickle.dump(cluster, f)
with open("nb4_msm.pkl", "wb") as f:
    pickle.dump(msm, f)
with open("nb4_bmsm.pkl", "wb") as f:
    pickle.dump(bayesian_msm, f)

## Case 2: low-dimensional molecular dynamics data (alanine dipeptide)

We fetch the alanine dipeptide data set, load the backbone torsions into memory,
directly discretize the full space using $k$-means clustering,
visualize the margial and joint distributions of both components as well as the cluster centers,
and show the ITS convergence to help selecting a suitable lag time:

In [ ]:
pdb = mdshare.fetch('alanine-dipeptide-nowater.pdb', working_directory='data')
files = mdshare.fetch('alanine-dipeptide-*-250ns-nowater.xtc', working_directory='data')

feat = pyemma.coordinates.featurizer(pdb)
feat.add_backbone_torsions(periodic=False)
data = pyemma.coordinates.load(files, features=feat)

cluster = KMeans(100).fit_fetch(np.concatenate(data))
dtrajs = [cluster.transform(traj) for traj in data]

lags = [1, 2, 5, 10, 20, 50]
its = implied_timescales([MaximumLikelihoodMSM(lagtime=lag).fit_fetch(dtrajs) for lag in lags])

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
pyemma.plots.plot_feature_histograms(
    np.concatenate(data), feature_labels=['$\Phi$', '$\Psi$'], ax=axes[0])
pyemma.plots.plot_density(*np.concatenate(data).T, ax=axes[1], cbar=False, alpha=0.1)
axes[1].scatter(*cluster.cluster_centers.T, s=15, c='C1')
axes[1].set_xlabel('$\Phi$')
axes[1].set_ylabel('$\Psi$')
plot_implied_timescales(its, ax=axes[2], n_its=4)
axes[2].set_yscale('log')
axes[2].set_xlabel('lagtime (ps)')
fig.tight_layout()

The plots show us the marginal (left panel) and joint distributions along with the cluster centers (middle panel).
The implied timescales are converged (right panel). 

We then estimate an MSM at lag time $10$ ps and visualize the stationary distribution by coloring all data points according to the stationary weight of the discrete state they belong to:

In [ ]:
msm = MaximumLikelihoodMSM(lagtime=10).fit_fetch(dtrajs)

print('fraction of states used = {:f}'.format(msm.state_fraction))
print('fraction of counts used = {:f}'.format(msm.count_fraction))

fig, ax, misc = pyemma.plots.plot_contour(
    *np.concatenate(data).T, msm.stationary_distribution[np.concatenate(dtrajs)],
    cbar_label='stationary_distribution',
    method='nearest', mask=True)
ax.scatter(*cluster.cluster_centers.T, s=15, c='C1')
ax.set_xlabel('$\Phi$')
ax.set_ylabel('$\Psi$')
fig.tight_layout()

Next, we visualize the first six right eigenvectors:

In [ ]:
eigvec = msm.eigenvectors_right()
print('first eigenvector is one: {} (min={}, max={})'.format(
    np.allclose(eigvec[:, 0], 1, atol=1e-15), eigvec[:, 0].min(), eigvec[:, 0].max()))

fig, axes = plt.subplots(2, 3, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    pyemma.plots.plot_contour(
        *np.concatenate(data).T, eigvec[np.concatenate(dtrajs), i + 1], ax=ax, cmap='PiYG',
        cbar_label='{}. right eigenvector'.format(i + 2), mask=True)
    ax.scatter(*cluster.cluster_centers.T, s=15, c='C1')
    ax.set_xlabel('$\Phi$')
    ax.set_ylabel('$\Psi$')
fig.tight_layout()

Again, we have the $(1,\dots,1)^\top$ first right eigenvector of the stationary process.

The second to fourth right eigenvectors illustrate the three slowest processes which are (in that order):

- rotation of the $\Phi$ dihedral,
- rotation of the $\Psi$ dihedral when $\Phi\approx-2$ rad, and
- rotation of the $\Psi$ dihedral when $\Phi\approx1$ rad.

Eigenvectors five, six, and seven indicate further processes which, however, relax faster than the lag time and cannot be resolved clearly.

We now proceed our validation process using a Bayesian MSM with four metastable states:

In [ ]:
nstates = 4

counts = TransitionCountEstimator(10, 'effective').fit_fetch(dtrajs)
counts = counts.submodel_largest()
bayesian_msm = BayesianMSM().fit_fetch(counts)
ck_test_bmsms = [
    BayesianMSM().fit_fetch(TransitionCountEstimator(lag, 'effective').fit_fetch(dtrajs).submodel_largest()) 
    for lag in tqdm([10*(i+1) for i in range(1, 10)])
]
ck_test = bayesian_msm.ck_test(ck_test_bmsms, nstates)
plot_ck_test(ck_test, xlabel='lagtime (ps)', sharey=True);

We note that four metastable states are a reasonable choice for our MSM.

In general, the number of metastable states is a modeler's choice; it is adjusted to map the kinetics to be modeled.
In the current example, increasing the resolution with a higher number of metastable states or resolving only the slowest process between $2$ states would be possible.
However, the number of states is not arbitrary as the observed processes in metastable state space need not be Markovian in general.
A failed Chapman-Kolmogorov test can thus also hint to a bad choice of the metastable state number.

In order to perform further analysis, we save the model to disk:

In [ ]:
with open('nb4_ala2_cluster.pkl', 'wb') as f:
    pickle.dump(cluster, f)
with open('nb4_ala2_msm.pkl', 'wb') as f:
    pickle.dump(msm, f)
with open('nb4_ala2_bayesian_msm.pkl', 'wb') as f:
    pickle.dump(bayesian_msm, f)

#### Exercise 1
Load the heavy atom distances into memory, TICA (`lag=3` and `dim=2`), discretize with  100 $k$-means centers and a stride of $10$, and show the ITS convergence.

In [ ]:
feat = #FIXME
feat. #FIXME
data = #FIXME

tica = #FIXME
projected_data = # FIXME

cluster = #FIXME
dtrajs = #FIXME

its_result = #FIXME

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
pyemma.plots.plot_feature_histograms(np.concatenate(projected_data), 
                                     feature_labels=['IC 1', 'IC 2'], ax=axes[0])
pyemma.plots.plot_density(*np.concatenate(projected_data).T, ax=axes[1], cbar=False, alpha=0.3)
axes[1].scatter(*cluster.cluster_centers.T, s=15, c='C1')
axes[1].set_xlabel('IC 1')
axes[1].set_ylabel('IC 2')
plot_implied_timescales(its_result, ax=axes[2])
ax.set_xlabel('lagtime (ps)')
ax.set_yscale('log')
fig.tight_layout()

###### Solution

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
pairs = feat.pairs(feat.select_Heavy())
feat.add_distances(pairs, periodic=False)
data = pyemma.coordinates.load(files, features=feat)

from deeptime.decomposition import TICA
tica_estimator = TICA(lagtime=3, dim=2)
tica = tica_estimator.fit_fetch(data)

projected_data = [tica.transform(traj) for traj in data]

cluster = KMeans(100).fit_fetch(np.concatenate(projected_data))
dtrajs = [cluster.transform(traj) for traj in projected_data]

its_result = implied_timescales([
    MaximumLikelihoodMSM(lagtime=lag).fit_fetch(dtrajs) for lag in [1, 2, 5, 10, 20, 50]
])

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
pyemma.plots.plot_feature_histograms(np.concatenate(projected_data), 
                                     feature_labels=['IC 1', 'IC 2'], ax=axes[0])
pyemma.plots.plot_density(*np.concatenate(projected_data).T, ax=axes[1], cbar=False, alpha=0.3)
axes[1].scatter(*cluster.cluster_centers.T, s=15, c='C1')
axes[1].set_xlabel('IC 1')
axes[1].set_ylabel('IC 2')
plot_implied_timescales(its_result, ax=axes[2], n_its=4)
axes[2].set_xlabel('lagtime (ps)')
axes[2].set_yscale('log')
fig.tight_layout()

#### Exercise 2
Estimate an MSM at lag time $10$ ps with `dt_traj='1 ps'` and visualize the stationary distribution using a two-dimensional colored scatter plot of all data points in TICA space.

In [ ]:
msm = #FIXME

print('fraction of states used = {:f}'. #FIXME
print('fraction of counts used = {:f}'. #FIXME

fig, ax, misc = pyemma.plots.plot_contour(
    *np.concatenate(projected_data).T, msm.stationary_distribution[np.concatenate(dtrajs)],
    cbar_label='stationary_distribution',
    method='nearest', mask=True)
ax.scatter(*cluster.cluster_centers.T, s=15, c='C1')
ax.set_xlabel('IC 1')
ax.set_ylabel('IC 2')
fig.tight_layout()

###### Solution

In [ ]:
msm = MaximumLikelihoodMSM(lagtime=10).fit_fetch(dtrajs)

print('fraction of states used = {:f}'.format(msm.state_fraction))
print('fraction of counts used = {:f}'.format(msm.count_fraction))

fig, ax, misc = pyemma.plots.plot_contour(
    *np.concatenate(projected_data).T, msm.stationary_distribution[np.concatenate(dtrajs)],
    cbar_label='stationary_distribution',
    method='nearest', mask=True)
ax.scatter(*cluster.cluster_centers.T, s=15, c='C1')
ax.set_xlabel('IC 1')
ax.set_ylabel('IC 2')
fig.tight_layout()

#### Exercise 3
Visualize the first six right eigenvectors.

In [ ]:
eigvec = #FIXME
print('first eigenvector is one: {} (min={}, max={})'.format( #FIXME

fig, axes = plt.subplots(2, 3, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    pyemma.plots.plot_contour(
        *np.concatenate(projected_data).T, eigvec[np.concatenate(dtrajs), i + 1], ax=ax, cmap='PiYG',
        cbar_label='{}. right eigenvector'.format(i + 2), mask=True)
    ax.scatter(*cluster.cluster_centers.T, s=15, c='C1')
    ax.set_xlabel('IC 1')
    ax.set_ylabel('IC 2')
fig.tight_layout()

###### Solution

In [ ]:
eigvec = msm.eigenvectors_right()
print('first eigenvector is one: {} (min={}, max={})'.format(
    np.allclose(eigvec[:, 0], 1, atol=1e-15), eigvec[:, 0].min(), eigvec[:, 0].max()))

fig, axes = plt.subplots(2, 3, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    pyemma.plots.plot_contour(
        *np.concatenate(projected_data).T, eigvec[np.concatenate(dtrajs), i + 1], ax=ax, cmap='PiYG',
        cbar_label='{}. right eigenvector'.format(i + 2), mask=True)
    ax.scatter(*cluster.cluster_centers.T, s=15, c='C1')
    ax.set_xlabel('IC 1')
    ax.set_ylabel('IC 2')
fig.tight_layout()

Can you already guess from eigenvectors two to four which the metastable states are?

#### Exercise 4
Estimate a Bayesian MSM at lag time $10$ ps and perform/show a CK test for four metastable states.

In [ ]:
bayesian_msm = #FIXME

nstates = 4
pyemma.plots. #FIXME

###### Solution

In [ ]:
counts = TransitionCountEstimator(10, 'effective').fit_fetch(dtrajs).submodel_largest()
bayesian_msm_estimator = BayesianMSM()
bayesian_msm = bayesian_msm_estimator.fit_fetch(counts.submodel_largest())

nstates = 4
bmsms = [
    BayesianMSM().fit_fetch(TransitionCountEstimator(lag, 'effective').fit_fetch(dtrajs).submodel_largest())
    for lag in tqdm(np.linspace(bayesian_msm.lagtime+1, 100, num=10, dtype=int))
]

ck_test = bayesian_msm.ck_test(bmsms, nstates)
plot_ck_test(ck_test, sharey=True, xlabel='lagtime (ps)');

#### Exercise 5
Save the MSM, Bayesian MSM and Cluster objects to the same file as before.
Use the model names `ala2tica_msm`, `ala2tica_bayesian_msm` and `ala2tica_cluster`, respectively.
Further, include the TICA object with model name `ala2tica_tica`.

In [ ]:
#FIXME 

###### Solution

In [ ]:
with open('nb4_ala2tica_cluster.pkl', 'wb') as f:
    pickle.dump(cluster, f)
with open('nb4_ala2tica_msm.pkl', 'wb') as f:
    pickle.dump(msm, f)
with open('nb4_ala2tica_bayesian_msm.pkl', 'wb') as f:
    pickle.dump(bayesian_msm, f)
with open('nb4_ala2tica_tica.pkl', 'wb') as f:
    pickle.dump(tica, f)

## Wrapping up
In this notebook, we have learned how to analyze an MSM and how to extract kinetic information from the model. 

In detail, we have used
- the `state_fraction`, `count_fraction`, and `count_model.state_symbols` attributes of an MSM object to see how much (and which parts) of our data form the largest connected set represented by the MSM,
- the `stationary_distribution` attribute of an MSM object to access its stationary vector,
- the `eigenvectors_right()` method of an MSM object to access its (right) eigenvectors,

For visualizing MSMs or kinetic networks we used
- `pyemma.plots.plot_density()`
- `pyemma.plots.plot_contour()` and
- `pyemma.plots.plot_cktest()`.